In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer

# Implemented by myself
from config import *
from data_processer import CSCDataset
from models import CombineBertModel, DecoderLSTM

In [2]:
def cal_err(raw_sentence, pred_sentence, corr_sentence):
    matrices = ["over_corr", "total_err", "true_corr"]
    char_level = {key: 0 for key in matrices}
    sent_level = {key: 0 for key in matrices}

    f1 = f2 = 0
    for i, c in enumerate(raw_sentence):
        pc, cc = pred_sentence[i], corr_sentence[i]

        if cc != c:
            char_level["total_err"] += 1
            char_level["true_corr"] += pc == cc
            f1 = 1
        elif pc != cc:
            char_level["over_corr"] += 1
            f2 = 1

    if f1:
        sent_level["true_corr"] += all(pred_sentence == corr_sentence)
        sent_level["total_err"] += f1
    sent_level["over_corr"] += f2

    return char_level, sent_level

In [3]:
def test(model, tokenizer, test_data_loader):
    model.eval()
    total_loss = 0
    matrices = ["over_corr", "total_err", "true_corr"]
    test_char_level = {key: 0 for key in matrices}
    test_sent_level = {key: 0 for key in matrices}

    with torch.no_grad():
        for batch in test_data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)

            loss = F.cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()

            t = torch.argmax(outputs, dim=-1)
            nt = t * attention_mask
            pred = tokenizer.batch_decode(nt, skip_special_tokens=True)

            for i in range(len(t)):
                char_level, sent_level = cal_err(input_ids[i], nt[i], labels[i])
                test_char_level = {
                    key: test_char_level[key] + v for key, v in char_level.items()
                }
                test_sent_level = {
                    key: test_sent_level[key] + v for key, v in sent_level.items()
                }
        print(total_loss / len(test_data_loader), test_char_level, test_sent_level)

In [4]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)
encoder_model = BertModel.from_pretrained(checkpoint)

# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

decoder_model = DecoderLSTM(
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

In [5]:
train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)
train_data_loader = DataLoader(
    train_dataset, num_workers=0, shuffle=True, batch_size=16
)

test_dataset = CSCDataset([SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer)
test_data_loader = DataLoader(test_dataset, num_workers=0, shuffle=True, batch_size=32)

preprocessing sighan dataset: 2339it [00:00, 36154.20it/s]
preprocessing sighan dataset: 100%|█████████████████████████████████████████████| 2339/2339 [00:00<00:00, 71696.72it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 41217.27it/s]
preprocessing sighan dataset: 100%|█████████████████████████████████████████████| 3437/3437 [00:00<00:00, 31171.37it/s]

共3437句，共170330字，最长的句子有258字


In [6]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

CombineBertModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [ ]:
from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(
        enumerate(train_data_loader),
        desc=f"Epoch:{epoch+1}/{epochs}",
        total=len(train_data_loader),
    )

    for i, batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device).type(torch.float)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, src_mask=attention_mask)
        logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

        # 反向传播在这，故labels不需要传入模型
        loss = F.cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

    t = torch.argmax(outputs, dim=-1)
    nt = t * attention_mask
    pred = tokenizer.batch_decode(nt, skip_special_tokens=True)
    # print(pred)
    # print(f"origin{tokenizer.batch_decode(labels, skip_special_tokens=True)}")

    for i, v in enumerate(nt):
        r, l = input_ids[i], labels[i]
        print(tokenizer.decode(r, skip_special_tokens=True))
        print(tokenizer.decode(v, skip_special_tokens=True))
        print(tokenizer.decode(l, skip_special_tokens=True))
        print(cal_err(r, v, l))

    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_data_loader)}")
    # test(model, tokenizer, test_data_loader)

Epoch:1/50: 100%|████████████████████████████████████████████████████████| 147/147 [01:33<00:00,  1.57it/s, loss=5.635]


假 設 是 我 們 ， 一 旦 知 道 我 們 的 一 舉 一 動 都 有 人 觀 茶 ， 我 們 會 有 怎 麼 樣 的 感 覺 ？
我 我 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的
假 設 是 我 們 ， 一 旦 知 道 我 們 的 一 舉 一 動 都 有 人 觀 察 ， 我 們 會 有 怎 麼 樣 的 感 覺 ？
({'over_corr': 32, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
我 下 次 回 家 的 時 候 一 定 會 清 他 ， 清 清 楚 楚 的 寫 下 來 一 些 蔡 的 做 法 ， 好 不 好 ？
我 我 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的
我 下 次 回 家 的 時 候 一 定 會 請 他 ， 清 清 楚 楚 的 寫 下 來 一 些 菜 的 做 法 ， 好 不 好 ？
({'over_corr': 29, 'total_err': 2, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
首 先 ， 我 要 說 明 他 為 什 麼 不 算 商 品 。 儘 管 卻 有 人 買 賣 牠 ， 我 想 那 種 行 為 是 不 對 的 。
我 我 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的
首 先 ， 我 要 說 明 牠 為 什 麼 不 算 商 品 。 儘 管 卻 有 人 買 賣 牠 ， 我 想 那 種 行 為 是 不 對 的 。
({'over_corr': 35, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
E

Epoch:2/50: 100%|████████████████████████████████████████████████████████| 147/147 [02:25<00:00,  1.01it/s, loss=5.612]


我 也 受 到 了 經 濟 的 影 響 ： 我 們 是 要 上 班 但 是 薪 水 便 少 了 。
我 我 我 我 ， ， ， ， ， 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的
我 也 受 到 了 經 濟 的 影 響 ： 我 們 是 要 上 班 但 是 薪 水 變 少 了 。
({'over_corr': 24, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
最 近 廠 長 裡 面 發 出 一 種 很 難 聞 的 味 道 ， 這 種 味 道 不 知 讓 我 們 村 裡 的 空 氣 汙 染 ， 而 還 讓 大 家 覺 得 害 怕 。
我 我 我 我 ， ， ， ， ， ， 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的
最 近 廠 長 裡 面 發 出 一 種 很 難 聞 的 味 道 ， 這 種 味 道 不 只 讓 我 們 村 裡 的 空 氣 汙 染 ， 而 還 讓 大 家 覺 得 害 怕 。
({'over_corr': 42, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
我 那 樣 子 玩 大 概 兩 三 個 月 ， 每 一 個 禮 拜 根 不 一 樣 的 人 出 來 玩 ， 所 以 中 文 就 慢 慢 的 變 成 很 好 的 都 講 得 很 流 利 ， 還 有 因 為 常 常 在 電 腦 聊 天 打 子 都 打 得 很 快 ， 所 以 納 的 時 候 很 快 樂 。
我 我 我 我 ， ， ， ， ， 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 ， ， 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的
我 那 樣 子 玩 大 概 兩 三 個 月 

Epoch:3/50: 100%|████████████████████████████████████████████████████████| 147/147 [02:39<00:00,  1.09s/it, loss=4.666]


...

Epoch:5/50: 100%|████████████████████████████████████████████████████████| 147/147 [02:47<00:00,  1.14s/it, loss=4.070]


因 為 表 哥 說 現 在 他 遇 到 的 問 題 是 一 個 大 考 驗 ， 所 以 他 應 該 必 須 很 正 面 地 面 對 這 個 大 考 驗 以 及 不 斷 的 努 力 去 解 決 它 。
因 為 了 了 ， 在 在 他 很 到 的 一 好 是 一 個 大 了 了 ， 所 以 他 一 很 了 來 很 了 人 ， 人 人 這 個 大 了 了 以 了 不 了 的 了 。 人 了 了 在 。
因 為 表 哥 說 現 在 他 遇 到 的 問 題 是 一 個 大 考 驗 ， 所 以 他 應 該 必 須 很 正 面 地 面 對 這 個 大 考 驗 以 及 不 斷 地 努 力 去 解 決 它 。
({'over_corr': 28, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
因 為 他 走 天 才 到 台 北 來 ， 所 以 他 不 知 道 怎 去 學 校 。
因 為 他 很 天 很 到 很 了 來 ， 所 以 他 不 很 一 了 了 去 了 了 。
因 為 他 昨 天 才 到 台 北 來 ， 所 以 他 不 知 道 怎 麼 去 學 校 。
({'over_corr': 8, 'total_err': 2, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
媒 體 上 整 天 在 提 出 這 種 觀 點 ， 批 評 主 婦 的 該 念 快 是 一 種 時 髦 ， 而 很 少 會 聽 到 好 處 根 樂 趣 。
很 一 上 很 天 在 了 了 這 很 了 了 ， 了 了 了 了 的 一 了 了 是 一 很 ， 了 ， 一 很 很 會 很 到 好 了 來 了 了 。
媒 體 上 整 天 在 提 出 這 種 觀 點 ， 批 評 主 婦 的 概 念 快 是 一 種 時 髦 ， 而 很 少 會 聽 到 好 處 跟 樂 趣 。
({'over_corr': 23, 'total_err': 2, 'true_corr': tensor(0, device='cuda:0')}, 

Epoch:6/50: 100%|████████████████████████████████████████████████████████| 147/147 [02:47<00:00,  1.14s/it, loss=3.267]


因 為 那 個 時 候 是 第 一 次 上 課 ， 所 以 他 看 起 來 跟 老 師 不 開 方 。
因 為 這 個 時 候 是 這 一 不 上 很 ， 所 以 他 看 了 來 跟 老 師 不 人 方 。
因 為 那 個 時 候 是 第 一 次 上 課 ， 所 以 他 看 起 來 跟 老 師 不 開 放 。
({'over_corr': 6, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
在 花 蓮 縣 去 年 一 位 特 殊 的 高 中 生 因 未 收 到 老 師 的 色 情 暴 力 懷 孕 ， 那 件 事 造 成 連 學 長 格 致 。
在 學 了 上 去 年 一 個 了 了 的 學 人 生 因 很 影 到 老 師 的 了 情 了 了 了 了 ， 這 很 事 影 好 上 學 大 了 了 。
在 花 蓮 縣 去 年 一 位 特 殊 的 高 中 生 因 為 受 到 老 師 的 色 情 暴 力 懷 孕 ， 那 件 事 造 成 連 學 長 革 職 。
({'over_corr': 19, 'total_err': 4, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 1, 'total_err': 1, 'true_corr': 0})
可 是 ， 能 有 多 少 人 知 道 現 在 北 極 冰 榮 化 的 很 嚴 重 ， 我 們 的 地 球 越 來 越 熱 ， 天 災 不 段 的 發 生 等 ， 原 因 何 在 呢 ？
可 是 ， 能 有 多 人 人 很 道 現 在 自 了 了 了 了 的 很 了 很 ， 我 們 的 地 家 自 來 自 了 ， 天 影 不 自 的 生 生 了 ， 學 因 了 在 了 ？
可 是 ， 能 有 多 少 人 知 道 現 在 北 極 冰 融 化 的 很 嚴 重 ， 我 們 的 地 球 越 來 越 熱 ， 天 災 不 斷 地 發 生 等 ， 原 因 何 在 呢 ？
({'over_corr': 18, 'total_err': 3, 'true_corr': tensor(0, device='cuda:0')}

Epoch:7/50:   3%|█▉                                                        | 5/147 [00:05<02:37,  1.11s/it, loss=2.917]